### TL; DR

This notebook tries to turn a comment into a dataframe and then apply detoxify to that dataframe to get our final result

In [ ]:
import pandas as pd
from detoxify import Detoxify
import time

In [ ]:
data_folder = './Data/' # all my data is in one folder 
path_comments='comments_glipse.csv.gz'

In [ ]:
df_comments = pd.read_csv(data_folder + path_comments, compression='gzip') 
df_comments

In [ ]:
dict_={'id': 'Ugx0iUXL0VOjEbUWkbV4AaABAg', 'author': 'Oliver Quinn', 'authorLink': '/channel/UCRMTnf9wtY-prkq2-X0Ehwg', 'authorThumb': 'https://yt3.ggpht.com/-UYZxBOkq7ok/AAAAAAAAAAI/AAAAAAAAAAA/SdLSYaSl4y8/s48-c-k-c0x00ffffff-no-rj-mo/photo.jpg\'', 'text': 'Good subject.. relates to everything. Thanks', 'likes': 0, 'time': '1 week ago', 'edited': False, 'timestamp': 1555070326054, 'hasReplies': False}

In [ ]:
tryout=pd.DataFrame([dict_])